In [1]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import warnings
warnings.filterwarnings("ignore")

sys.path.append('../../')

#The following two lines are the openABC imports relevant for NEAT-DNA simulations.
from openabc.forcefields.parsers import NEATDNAParser
from openabc.forcefields.parsers import MOFFParser
from openabc.forcefields import MOFFNEATDNAModel

In this example we simulate the 1KX5 nucleosome structure with the NEAT-DNA and MOFF protein model. We provide a simple way to parse the nucleosome topology and define all necessary interactions. Use class NEATDNAParser to parse dsDNA and the MOFFParser for protein. Either an atomistic structure may be passed and automatically coarse-grained, or an already coarse-grain structure may be utilized.

In [2]:
# parse DNA and histone topology with atomistic model as input
dna = NEATDNAParser.from_atomistic_pdb('input-structures/1kx5.pdb', 'cg_DNA.pdb')
histone = MOFFParser.from_atomistic_pdb('input-structures/1KX5.pdb', 'cg_histone.pdb')

Parse molecule with default settings.
Get native pairs with shadow algorithm.


Below we define a function to select atom indices of the histone core and a specified number of DNA base pairs centered around the dyad to set rigid bodies for simulation.

In [3]:
def get_chromatin_rigid_bodies(nucleosome_center_positions, trailing_bps=73, n_rigid_bp_per_nucleosome=73, fully_rigid_histone=False):
    _histone_tail_start_atoms = np.array([1, 136, 238, 353, 488, 623, 725, 840]) - 1
    _histone_tail_end_atoms = np.array([43, 159, 257, 400, 530, 646, 744, 887]) - 1
    _histone_tail_atoms = []
    for i in range(8):
        _histone_tail_atoms += list(range(_histone_tail_start_atoms[i], _histone_tail_end_atoms[i] + 1))
    _histone_tail_atoms = np.array(_histone_tail_atoms)

    _n_CA_atoms_per_histone = 974

    _histone_core_atoms = np.array([x for x in range(_n_CA_atoms_per_histone) if x not in _histone_tail_atoms])
    if fully_rigid_histone:
        _histone_core_atoms = np.array([x for x in range(_n_CA_atoms_per_histone)])

    _n_bp_per_nucl = 147
    n_histone_atoms = len(nucleosome_center_positions)*_n_CA_atoms_per_histone
    n_dna_atoms = 2*(_n_bp_per_nucl)

    rigid_bodies = []
    dna_rigid = []
    for i in range(len(nucleosome_center_positions)):
        rigid_body = []
        rigid_body += (_histone_core_atoms + i*_n_CA_atoms_per_histone).tolist()

        start_bp_id = nucleosome_center_positions[i] - int(n_rigid_bp_per_nucleosome/2)
        end_bp_id = start_bp_id + n_rigid_bp_per_nucleosome
        for j in range(start_bp_id, end_bp_id):
            rigid_body += [n_histone_atoms + j]
            rigid_body += [n_histone_atoms + n_dna_atoms - j - 1]
            dna_rigid += [n_histone_atoms + j]
            dna_rigid += [n_histone_atoms + n_dna_atoms - j - 1]

        rigid_bodies += [sorted(rigid_body)]
    return rigid_bodies

Here we append the parsed histone and dna, set up the system and add all relevant forces. Alternatively, you can call:

`model.add_all_default_forces(salt_concentration,temperature)`

Which will add the same set of interactions after appending the molecules.

In [4]:
model = MOFFNEATDNAModel()
model.append_mol(histone)
model.append_mol(dna)
model.atoms_to_pdb("cg_nucleosome.pdb")
top = app.PDBFile("cg_nucleosome.pdb").getTopology()
init_coord = app.PDBFile("cg_nucleosome.pdb").getPositions()

salt_concentration = 150*unit.millimolar
temperature = 300*unit.kelvin

model.create_system(top, use_pbc=False)
model.add_protein_bonds(force_group=1)
model.add_protein_angles(force_group=2)
model.add_dna_bonds(force_group=3)
model.add_dna_angles(force_group=4)
model.add_dna_breakable_fan_bonds(force_group=5)
model.add_elec(salt_conc=salt_concentration, temperature=temperature, force_group=6)
model.add_dna_bp_bonds(force_group=7)
model.add_dna_bp_dihedrals(force_group=8)
model.add_dna_bp_angles(force_group=9)
model.add_contacts(force_group=10)
model.add_native_pairs(force_group=11)

Add protein bonds.
Add DNA Bonds.
Add DNA Angles.
Add DNA Breakable Fan Bonds.
Add DNA electrostatic interactions with constant dielectric, no-switch, and 0.6 correction.
Add DNA Base-Pairing Bonds.
Add DNA Base-Pairing Dihedrals.
Add DNA Base-Pairing Angles.
Add protein and DNA nonbonded contacts.
Add native pairs.


We then use the previously defined function to set rigid bodies to keep the histone core rigid and keep the DNA fixed to dyad.

In [5]:
#Get atoms indices for rigid bodies
rigid_bodies = get_chromatin_rigid_bodies([73],n_rigid_bp_per_nucleosome=5)
model.set_rigid_bodies(init_coord, rigid_bodies)

model.save_system("nucleosome.xml")

Finally, we setup and run a short NVT simulation

In [6]:
friction_coeff = 1.0 / unit.picosecond
timestep = 10 * unit.femtosecond
platform_name = 'CPU'
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
model.set_simulation(integrator, platform_name, init_coord=init_coord)
model.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output.dcd'
model.add_reporters(output_interval, output_dcd)
model.simulation.context.setVelocitiesToTemperature(temperature)
model.simulation.step(500)

Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,23894.071962264687,1795.1048089837925,25689.17677124848,259.341120630635,0
200,2.0000000000000013,24197.79859218804,2081.0175329091167,26278.816125097157,300.64730278460434,95.5
300,2.99999999999998,24176.241830252482,2026.4368959281944,26202.678726180675,292.76196734985643,95.7
400,3.9999999999999587,24210.523396645942,2038.726656796942,26249.250053442884,294.53748504666873,96.8
500,4.999999999999938,24331.74106107429,2071.6080729245987,26403.34913399889,299.28790589329105,96.8
